In [1]:
import urllib.request
import json

In [2]:
import urllib.request
import urllib.parse
import json
import pprint

class CoreApiRequestor:

    def __init__(self, endpoint, api_key):
        self.endpoint = endpoint
        self.api_key = api_key
        #defaults
        self.pagesize = 100
        self.page = 1

    def parse_response(self, decoded):
        res = []
        for item in decoded['data']:
            doi = None
            if 'identifiers' in item:
                for identifier in item['identifiers']:
                    if identifier and identifier.startswith('doi:'):
                        doi = identifier
                        break
            res.append([item['title'], doi])
        return res

    def request_url(self, url):
        with urllib.request.urlopen(url) as response:
            html = response.read()
        return html

    def get_method_query_request_url(self,method,query,fullText,page):
        if (fullText):
            fullText = 'true'
        else:
            fullText = 'false'
        params = {
            'apiKey':self.api_key,
            'page':page,
            'pageSize':self.pagesize,
            'fulltext':fullText,
            'urls': 'true'
        }
        return self.endpoint + method + '/' + urllib.parse.quote(query) + '?' + urllib.parse.urlencode(params)

    def get_up_to_20_pages_of_query(self,method,query,fulltext):
        url = self.get_method_query_request_url(method,query,fulltext,1)
        all_articles=[]
        resp = self.request_url(url)
        result = json.loads(resp.decode('utf-8'))
        all_articles.append(result)
        if (result['totalHits']>100):
            numOfPages = int(result['totalHits']/self.pagesize)  #rounds down
            if (numOfPages>20):
                numOfPages=20
            for i in range(2,numOfPages):
                url = self.get_method_query_request_url(method,query,False,i)
                print(url)
                resp =self.request_url(url)
                all_articles.append(json.loads(resp.decode('utf-8')))
        return all_articles

In [3]:
endpoint = 'https://core.ac.uk/api-v2'

api_key = '5TphODMbCjoQUkWitNZSBVqsEeYlHfnI'

method = '/articles/search'
topic = '*'

api = CoreApiRequestor(endpoint, api_key)

In [4]:
url = api.get_method_query_request_url(method, topic, False, 1)
url

'https://core.ac.uk/api-v2/articles/search/%2A?apiKey=5TphODMbCjoQUkWitNZSBVqsEeYlHfnI&page=1&pageSize=100&fulltext=false&urls=true'

In [5]:
result = api.get_up_to_20_pages_of_query(method, topic, False)
j = json.loads(result.decode('utf-8'))
d = j['data']
d

https://core.ac.uk/api-v2/articles/search/%2A?apiKey=5TphODMbCjoQUkWitNZSBVqsEeYlHfnI&page=2&pageSize=100&fulltext=false&urls=true
https://core.ac.uk/api-v2/articles/search/%2A?apiKey=5TphODMbCjoQUkWitNZSBVqsEeYlHfnI&page=3&pageSize=100&fulltext=false&urls=true
https://core.ac.uk/api-v2/articles/search/%2A?apiKey=5TphODMbCjoQUkWitNZSBVqsEeYlHfnI&page=4&pageSize=100&fulltext=false&urls=true
https://core.ac.uk/api-v2/articles/search/%2A?apiKey=5TphODMbCjoQUkWitNZSBVqsEeYlHfnI&page=5&pageSize=100&fulltext=false&urls=true
https://core.ac.uk/api-v2/articles/search/%2A?apiKey=5TphODMbCjoQUkWitNZSBVqsEeYlHfnI&page=6&pageSize=100&fulltext=false&urls=true
https://core.ac.uk/api-v2/articles/search/%2A?apiKey=5TphODMbCjoQUkWitNZSBVqsEeYlHfnI&page=7&pageSize=100&fulltext=false&urls=true
https://core.ac.uk/api-v2/articles/search/%2A?apiKey=5TphODMbCjoQUkWitNZSBVqsEeYlHfnI&page=8&pageSize=100&fulltext=false&urls=true
https://core.ac.uk/api-v2/articles/search/%2A?apiKey=5TphODMbCjoQUkWitNZSBVqsEeYlHf

AttributeError: 'list' object has no attribute 'decode'

In [6]:
print(type(result[0]))
print(result[0]['data'][0]['title'])

<class 'dict'>
Escola catalana


In [7]:
result[0]['data'][99]

{'id': '227381854',
 'authors': ['Huisman, Drs. JJ (ProRail bv)'],
 'contributors': ['van der A, Drs. SJH (ProRail bv)'],
 'datePublished': '20170626-01-01T00:00:00.000+01:00',
 'description': 'Een archeologisch bureauonderzoek t.b.v. de voorgenomen wijziging van het opstelterrein te Lelystad',
 'identifiers': ['oai:easy.dans.knaw.nl:easy-dataset:73135',
  '10.17026/dans-xge-a3yq'],
 'publisher': 'ProRail bv',
 'relations': [],
 'repositories': [{'id': '5577',
   'openDoarId': 0,
   'name': 'Electronic Archiving System',
   'uri': None,
   'urlHomepage': None,
   'urlOaipmh': None,
   'uriJournals': None,
   'physicalName': 'noname',
   'source': None,
   'software': None,
   'metadataFormat': None,
   'description': None,
   'journal': None,
   'roarId': 0,
   'baseId': 0,
   'pdfStatus': None,
   'nrUpdates': 0,
   'disabled': False,
   'lastUpdateTime': None,
   'repositoryLocation': None}],
 'repositoryDocument': {'pdfStatus': 0,
  'metadataAdded': 1567968358000,
  'metadataUpdated

In [8]:
def clean(result):
    mega_json = []
    for p in result:
        for a in p['data']:
            try:
                obj = {}
                k = ['authors', 'title', 'description', 'datePublished', 'fulltextUrls', 'journal']
                for i in k:
                    if i in a:
                        obj[i] = a[i]
                if 'fulltextUrls' in obj:
                    obj['url'] = obj.pop('fulltextUrls')
#             print(a['title'])
                mega_json.append(obj)
            except:
                print('try again')
    return mega_json

cleaned_data = clean(result)

In [9]:
cleaned_data
len(cleaned_data)

1900

In [10]:
from elasticsearch import Elasticsearch

In [11]:
es = Elasticsearch()

for doc in range(len(cleaned_data)):
    if doc == 100:
        break
    res = es.index(index="core", id=doc, body=cleaned_data[doc])
    print(doc)

res = es.get(index="core", id=2)
print(res['_source'])

es.indices.refresh(index="core")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
{'authors': ['Sinae RYOO'], 'title': 'Spring Orientation (15)', 'datePublished': '20190129-0131', 'url': ['https://core.ac.uk/display/213854370', 'https://archives.kdischool.ac.kr/handle/11125/31058']}


{'_shards': {'total': 2, 'successful': 1, 'failed': 0}}

In [81]:
res = es.get(index="core", id=10)
print(res['_source'])
# es.indices.refresh(index='test-index')

{'authors': [], 'title': "Fonds d'archives Michel Jaffrennou. Archives Patrick Bousquet. Négatifs, planches contacts et diapositives. Oeuvres de Charlotte Moorman et Nam June Paik (planche-contact)", 'description': "Numérisation effectuée à partir d'un document original : DONAUD1409 -145.Voir les tirages réalisées d'après le même négatif sous la cote PHO DONAUD1409-131. Négatif sous la cote DONAUD1409 -145.", 'datePublished': '19770101-19831231', 'url': ['https://core.ac.uk/display/202057447', 'http://gallica.bnf.fr/ark:/12148/btv1b53184848m', 'http://archivesetmanuscrits.bnf.fr/ark:/12148/cc103092g/ca19943598']}
